In [ ]:
import json

import altair as alt
import pandas as pd
 
with open("grpo2.jsonl", "r") as f:
    lines = f.readlines()

stats = []
for i, line in enumerate(lines):
    data = json.loads(line)
    if data["run_id"] not in {
        # "6df3c011",
        # "5c172e2b",
        # "34f149c3",
        # lora_rank=16!
        # "6b6e34e9",
        # computation graph reward shaping
        # "3294e8e9",
        # temp=0.9, lora_rank=16, computation graph reward shaping
        # "c5b79e89",
        # temp=1.2, lora_rank=16, minimal rewards
        # "c15c5ae6",
        # "0117ec79",
        # 'fc929682',
        "c6cfce32",
        "3caa4bed",

        # SFT
        # "c3ad3436",
        # "431e7d9a",
        "6a492447",
        
        # SFT + computation graph
        "ae4093e3",
        "a29a3777",
    }: continue
    for key in data.keys():
        if key.startswith("reward"):
            value = data[key]
            stats.append({
                "step": data["step"],
                "id": i,
                "run_id": data["run_id"],
                "value": value, "reward": key,
            })

df = pd.DataFrame(stats)

alt.data_transformers.enable("vegafusion")

bins = 128
mean = alt.Chart(df).encode(
    alt.X('step:Q').bin(maxbins=bins),
    alt.Y('mean(value):Q'),
    alt.Color('run_id:N'),
).mark_line()

errorband = alt.Chart(df).encode(
    alt.X('step:Q').bin(maxbins=bins),
    alt.Y('max(value):Q'),
    alt.Color('run_id:N'),
).mark_errorband(extent="ci")

maximum = alt.Chart(df).encode(
    alt.X('step:Q').bin(maxbins=bins),
    alt.Y('max(value):Q'),
    alt.Color('run_id:N'),
).mark_circle()

(errorband + mean + maximum).facet("reward")

In [ ]:
df.groupby(["reward", "run_id"])[["value"]].max()